TODO: change to use own model instead of pretrained

In [20]:
from pathlib import Path
import yaml
from pyannote.audio import Pipeline, Model
import os
from dotenv import load_dotenv
import re
from pydub import AudioSegment
from pyannote.audio.pipelines import SpeakerDiarization
from natsort import natsorted

load_dotenv()

huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

In [2]:
trained_pipeline = Pipeline.from_pretrained(Path("./pipeline/config.yaml"))
model = Model.from_pretrained("./models/trained_model.ckpt")
trained_pipeline.segmentation_model = model

with open("./parameters/config.yaml", "r") as f:
    parameters = yaml.safe_load(f)

params = parameters["params"]

params_clust = params["clustering"]

trained_pipeline.instantiate({
        "segmentation": {"min_duration_off": params["segmentation"]["min_duration_off"]},
        "clustering": params["clustering"]
})

trained_pipeline.dump_params(Path('./parameters/temp.yaml'))

'params:\n  clustering:\n    method: centroid\n    min_cluster_size: 15\n    threshold: 0.7097170356971093\n  segmentation:\n    min_duration_off: 0.0\n'

In [3]:
pipeline = SpeakerDiarization(
    segmentation=model,
    embedding=trained_pipeline.embedding,
    embedding_exclude_overlap=trained_pipeline.embedding_exclude_overlap,
    clustering=trained_pipeline.klustering,
)

with open("./parameters/config.yaml", "r") as f:
    loaded_parameters = yaml.safe_load(f)

loaded_params = loaded_parameters["params"]

params_clust = loaded_params["clustering"]

parameters_segm = parameters["params"]["segmentation"]


pipeline.instantiate({
    "segmentation": {
        "min_duration_off": parameters_segm["min_duration_off"],
        "threshold": parameters_segm["threshold"]
    },
    "clustering": params_clust
})

# trained_pipeline.dump_params(Path('./parameters/temp.yaml'))

In [4]:
# TODO: niet hardgecodeerd voor eindresultaat
path = "../audio-fragmenten/audio/BP_elderspeak_01-03-2024/1-03-lab1.wav"

In [5]:
# apply pretrained pipeline
diarization = pipeline(path)    

/home/lisa_verhoeyen/.local/lib/python3.10/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


create rttm file based on model result

In [6]:
file_name = re.search(r'.*\/(.+?)\.wav', path).group(1)

# delete file if it already exists so a new file can get created
if os.path.exists(f"./results/rttm/{file_name}.rttm"):
    os.remove(f"./results/rttm/{file_name}.rttm")

f = open(f"./results/rttm/{file_name}.rttm", "w")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    f.write(f"SPEAKER {file_name} 1 {round(turn.start, 3)} {round(turn.end-turn.start, 3)} <NA> <NA> {speaker} <NA> <NA>\n")

f.close()

turn result into audio files

In [ ]:
audio = AudioSegment.from_wav(f"../audio-fragmenten/audio/BP_elderspeak_01-03-2024/{file_name}.wav")

# if os.path.exists(f"./results/audio/{file_name}/"):
#     os.rmdir(f"./results/audio/{file_name}/")

os.mkdir(f"./results/audio/{file_name}/")


for turn, _, speaker in diarization.itertracks(yield_label=True):
    if not os.path.isdir(f"./results/audio/{file_name}/{speaker}"):
        os.mkdir(f"./results/audio/{file_name}/{speaker}")

    segment = audio[turn.start * 1000:turn.end * 1000]  # Convert seconds to milliseconds
    segment.export(f"./results/audio/{file_name}/{speaker}/speaker_{speaker}_{turn.start:.1f}-{turn.end:.1f}.wav", format="wav")

create audio file based on speaker by combining audio files

In [ ]:
speakers = os.listdir(f"./results/audio/{file_name}")

for speaker in speakers:
    files_list = os.listdir(f"./results/audio/{file_name}/{speaker}")
    files_list = natsorted(files_list)

    combined = AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[0]}")
    for i in range(1,len(files_list)):
        combined = combined.append(AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[i]}"), crossfade=0.0)

    combined.export(f"./results/audio/{file_name}_{speaker}.wav", format="wav")

['speaker_SPEAKER_00_0.0-0.1.wav', 'speaker_SPEAKER_00_14.1-15.4.wav', 'speaker_SPEAKER_00_15.4-15.4.wav', 'speaker_SPEAKER_00_16.1-16.1.wav', 'speaker_SPEAKER_00_16.2-17.3.wav', 'speaker_SPEAKER_00_27.3-28.1.wav', 'speaker_SPEAKER_00_30.2-32.1.wav', 'speaker_SPEAKER_00_35.2-35.8.wav', 'speaker_SPEAKER_00_36.6-36.6.wav', 'speaker_SPEAKER_00_113.4-113.6.wav', 'speaker_SPEAKER_00_122.3-122.4.wav', 'speaker_SPEAKER_00_127.8-128.5.wav', 'speaker_SPEAKER_00_128.7-129.0.wav', 'speaker_SPEAKER_00_131.5-132.4.wav', 'speaker_SPEAKER_00_135.1-135.8.wav', 'speaker_SPEAKER_00_140.5-140.5.wav', 'speaker_SPEAKER_00_140.5-140.6.wav', 'speaker_SPEAKER_00_140.7-140.7.wav', 'speaker_SPEAKER_00_149.9-150.6.wav', 'speaker_SPEAKER_00_150.6-150.6.wav', 'speaker_SPEAKER_00_153.8-155.7.wav', 'speaker_SPEAKER_00_161.1-162.0.wav', 'speaker_SPEAKER_00_164.0-166.1.wav', 'speaker_SPEAKER_00_166.2-166.3.wav', 'speaker_SPEAKER_00_166.3-166.4.wav', 'speaker_SPEAKER_00_166.4-166.4.wav', 'speaker_SPEAKER_00_166.6-167.2

## Testing stuff